In [1]:
import spotipy
import credentials as cr
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

## Application parameters

In [ ]:
# redirect_uri = 'http://localhost/'
client_id = cr.CLIENT_ID
client_secret = cr.CLIENT_SECRET
auth = cr.TOKEN
data_path = cr.DATA_PATH

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, auth=auth)

In [ ]:
# def get_features(track_id):

#     return sp.audio_features(track_id)

In [ ]:
def get_user_playlists():
    """Get the current user's spotify playlists with a limit at 50.
    returns: pandas.DataFrame playlists"""

    # Get json playlists using spotipy api
    playlistsJson = sp.current_user_playlists(limit=50, offset=0)

    playlists = {'id': [], 'name': [], 'total_tracks': [], 'owner_id': [], 'href': []}
    
    while playlistsJson:
      for i, playlist in enumerate(playlistsJson['items']):
        playlists['id'].append(playlist.id)
        playlists['name'].append(playlist.name)
        playlists['total_tracks'].append(playlist.tracks.total)
        playlists['owner_id'].append(playlist.owner.id)
        playlists['href'].append(playlist.href)

    # Set dict as a dataframe object and returns it
    return pd.DataFrame(playlists).set_index('id', inplace=True)

In [ ]:
# Récupération des playlists

playlists_df = get_user_playlists()

playlists_df.head()
#nettoyage
#en csv

In [ ]:
# Récupération des chansons d'une playlist

#en liste
#en dataframe
#nettoyage
#en csv

In [ ]:
# méthod qui va appliquer la métode de sélection des chansons d'une playlist, appliquée à toutes les playlists d'un user